In [1]:
import time
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import squarify
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import re
from wordcloud import WordCloud

## Data cleaning.

Cleaning the 311 Requests dataset first.

In [3]:
# import parquet file
#req_df = pd.read_parquet("capstone_data.parquet.gzip")

Only keeping columns relevant to request type, location and resolution time.

In [4]:
req_df = req_df[['created_date', 'agency', 'agency_name', 'complaint_type', 'incident_zip', 'borough', 'community_board',
                 'resolution_description', 'closed_date', 'due_date', 'descriptor']]

Excluding rows with invalid ZIP codes, since it'll later be used to join median income data.

In [5]:
req_df = req_df[req_df['incident_zip'].notna()]
req_df = req_df[req_df['incident_zip'] != "NA"]

In [6]:
# looking at total amount of duplicates
req_df.duplicated().sum()

27315

In [7]:
# not a significant amount, we can drop duplicates
req_df.drop_duplicates(inplace=True)

In [8]:
req_df.count()

created_date              2936143
agency                    2936143
agency_name               2936143
complaint_type            2936143
incident_zip              2936143
borough                   2936143
community_board           2936143
resolution_description    2838451
closed_date               2768004
due_date                    15443
descriptor                2911135
dtype: int64

In [9]:
# converting date columns from object to datetime for easier manipulation
req_df['created_date']= pd.to_datetime(req_df['created_date'])
req_df['closed_date']= pd.to_datetime(req_df['closed_date'])
req_df['due_date']= pd.to_datetime(req_df['due_date'])

In [10]:
# converting columns from object to string and int for easier manipulation
req_df['agency'] = req_df['agency'].astype("string")
req_df['agency_name'] = req_df['agency_name'].astype("string")
req_df['complaint_type'] = req_df['complaint_type'].astype("string")
req_df['resolution_description'] = req_df['resolution_description'].astype("string")
req_df['borough'] = req_df['borough'].astype("string")
req_df['community_board'] = req_df['community_board'].astype("string")
req_df['descriptor'] = req_df['descriptor'].astype("string")
req_df['incident_zip'] = req_df['incident_zip'].astype("string").astype("int")

In [11]:
# sorting and reseting index to have a chronological ordered dataset
req_df = req_df.sort_values(by=['created_date'])
req_df = req_df.reset_index(drop=True)

In [12]:
req_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936143 entries, 0 to 2936142
Data columns (total 11 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   created_date            datetime64[ns]
 1   agency                  string        
 2   agency_name             string        
 3   complaint_type          string        
 4   incident_zip            int64         
 5   borough                 string        
 6   community_board         string        
 7   resolution_description  string        
 8   closed_date             datetime64[ns]
 9   due_date                datetime64[ns]
 10  descriptor              string        
dtypes: datetime64[ns](3), int64(1), string(7)
memory usage: 246.4 MB


In [13]:
req_df.shape

(2936143, 11)

In [14]:
req_df.head(1)

,created_date,agency,agency_name,complaint_type,incident_zip,borough,community_board,resolution_description,closed_date,due_date,descriptor
0,2023-06-10 21:11:28,NYPD,New York City Police Department,Non-Emergency Police Matter,10465,BRONX,10 BRONX,The Police Department responded to the complai...,2023-06-10 21:27:23,NaT,Other (complaint details)


Our request information includes the created date, the agency tasked with resolution, name of the agency, the complaint type, the ZIP code where the incident was reported, the Borough this ZIP code is a part of, the Community board this ZIP code is a part of, the description of the resolution, the date it was closed/resolved if any, due date of the resolution if any, and the description of the complaint if any.

---

Cleaning the median household income data.

In [15]:
# importing median income data
income_df = pd.read_csv("median_income.csv")

In [16]:
income_df.duplicated().sum()

0

In [17]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   zip                 180 non-null    int64
 1   median_income_2022  180 non-null    int64
dtypes: int64(2)
memory usage: 2.9 KB


In [18]:
# no invalid or duplicated values, we only need to change column names to match 311 requests dataset
income_df = income_df.rename(columns={"zip":"incident_zip"})

After cleaning, median household income dataset has 180 rows with 2 columns.

---

Cleaning poverty rate data.

In [19]:
# importing poverty rate data
poverty_df = pd.read_csv("poverty_rate.csv")

In [20]:
# only keep location and poverty rate related columns
poverty_df = poverty_df[["Sub-Borough Area", "2021"]]

In [21]:
poverty_df.duplicated().sum()

0

In [22]:
poverty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sub-Borough Area  55 non-null     object 
 1   2021              55 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1012.0+ bytes


In [23]:
poverty_df['Sub-Borough Area'] = poverty_df['Sub-Borough Area'].astype("string")

In [24]:
# no invalid or duplicated values, we only need to change column names for easier manipulation
poverty_df = poverty_df.rename(columns={"Sub-Borough Area":"sub_borough_area", "2021":"poverty_rate_2021"})

After cleaning, poverty rate dataset has 55 rows with 2 columns.

---

Cleaning racial diversity index data.

In [25]:
# importing racial diversity index data
diversity_df = pd.read_csv("racial_diversity_index.csv")

In [26]:
# only keep location and diversity index related columns
diversity_df = diversity_df[["Sub-Borough Area", "2021"]]

In [27]:
diversity_df.duplicated().sum()

0

In [28]:
diversity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sub-Borough Area  55 non-null     object 
 1   2021              55 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1012.0+ bytes


In [29]:
# no invalid or duplicated values, we only need to change column names for easier manipulation
diversity_df = diversity_df.rename(columns={"Sub-Borough Area":"sub_borough_area", "2021":"diversity_index_2021"})

After cleaning, diversity index dataset has 55 rows with 2 columns.

---

## Data preprocessing.

First, I'll join the main 311 Requests dataset with the relevant columns extracted from the racial diversity index data, poverty rate data, and median household income data.

In [30]:
# we can easily join these 2 datasets, using the many-to-one validation parameter to double-check we only get one value per ZIP code
req_df = req_df.merge(income_df, on="incident_zip", how="left", validate="m:1")

The closest location attribute that can be used from the 311 Requests dataset to join with  the racial diversity index and poverty rate datasets is the ```community_board``` column. These can be joined to each corresponding ```sub_borough_area```, given community board divisions closely resemble sub-borough divisions.

Community boards in NYC are advisory boards made up of community members. They make important recommendations about land use, zoning, and other local issues. These are comprised of multiple neighborhoods, and by using NYC OpenData's ["NYC Community Boards"](https://data.cityofnewyork.us/City-Government/NYC-Community-Boards/ruf7-3wgc/about_data) dataset and this [NYC Sub-Borough map](https://github.com/heriberto00/Capstone/blob/main/data/datasets/subboro.pdf), I was able to find and assign the corresponding ```community_board``` to each ```sub_borough_area``` in the racial diversity index and poverty rate datasets using the ```neighborhoods``` value in the community boards dataset.

To deal with the missing link between datasets, we can manually assign each sub-borough the closest corresponding community board.

In [31]:
poverty_df['community_board'] = ''

In [32]:
# manually assigning each community_board depending on sub_borough_area
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Greenwich Village/Financial District'] = '02 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Lower East Side/Chinatown'] = '03 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Kingsbridge Heights/Mosholu'] = '07 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Riverdale/Kingsbridge'] = '08 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Soundview/Parkchester'] = '09 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Throgs Neck/Co-op City'] = '10 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Pelham Parkway'] = '11 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Williamsbridge/Baychester'] = '12 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Williamsburg/Greenpoint'] = '01 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Brooklyn Heights/Fort Greene'] = '02 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Bedford Stuyvesant'] = '03 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Bushwick'] = '04 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'East New York/Starrett City'] = '05 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Park Slope/Carroll Gardens'] = '06 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Sunset Park'] = '07 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'North Crown Heights/Prospect Heights'] = '08 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Elmhurst/Corona'] = '04 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Middle Village/Ridgewood'] = '05 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Rego Park/Forest Hills'] = '06 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Flushing/Whitestone'] = '07 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Hillcrest/Fresh Meadows'] = '08 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Ozone Park/Woodhaven'] = '09 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'South Ozone Park/Howard Beach'] = '10 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Bayside/Little Neck'] = '11 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Jamaica'] = '12 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Queens Village'] = '13 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Rockaways'] = '14 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'North Shore'] = '01 STATEN ISLAND'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Mid-Island'] = '02 STATEN ISLAND'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'South Shore'] = '03 STATEN ISLAND'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Chelsea/Clinton/Midtown'] = '04 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Stuyvesant Town/Turtle Bay'] = '06 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Upper West Side'] = '07 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Upper East Side'] = '08 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Morningside Heights/Hamilton Heights'] = '09 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Central Harlem'] = '10 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'East Harlem'] = '11 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Washington Heights/Inwood'] = '12 MANHATTAN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Mott Haven/Hunts Point'] = '01 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Morrisania/Belmont'] = '02 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Highbridge/South Concourse'] = '04 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'University Heights/Fordham'] = '05 BRONX'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'South Crown Heights'] = '09 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Bay Ridge'] = '10 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Bensonhurst'] = '11 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Borough Park'] = '12 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Coney Island'] = '13 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Flatbush'] = '14 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Sheepshead Bay/Gravesend'] = '15 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Brownsville/Ocean Hill'] = '16 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'East Flatbush'] = '17 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Flatlands/Canarsie'] = '18 BROOKLYN'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Astoria'] = '01 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Sunnyside/Woodside'] = '02 QUEENS'
poverty_df['community_board'].loc[poverty_df['sub_borough_area'] == 'Jackson Heights'] = '03 QUEENS'

In [33]:
# merge diversity_df with poverty_df to get already assigned community_board values
diversity_df = diversity_df.merge(poverty_df[['sub_borough_area', 'community_board']], on="sub_borough_area", how="left", validate="m:1")

In [34]:
# manually adding index for 05 MANHATTAN community board because of overlapping neighborhoods
diversity_df = diversity_df[['diversity_index_2021', 'community_board']]
man05 = pd.DataFrame({'diversity_index_2021':[0.584941], 'community_board': ['05 MANHATTAN']})
diversity_df = diversity_df._append(man05, ignore_index = True)

In [35]:
# manually adding rate for 05 MANHATTAN community board because of overlapping neighborhoods
poverty_df = poverty_df[['poverty_rate_2021', 'community_board']]
man05 = pd.DataFrame({'poverty_rate_2021':[0.111038], 'community_board': ['05 MANHATTAN']})
poverty_df = poverty_df._append(man05, ignore_index = True)

In [36]:
# merge dataframes with main dataframe containing 311 requests
req_df = req_df.merge(poverty_df[['poverty_rate_2021','community_board']], on="community_board", how="left", validate="m:1")
req_df = req_df.merge(diversity_df[['diversity_index_2021','community_board']], on="community_board", how="left", validate="m:1")

In [37]:
req_df['community_board'] = req_df['community_board'].astype("string")

In [38]:
req_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936143 entries, 0 to 2936142
Data columns (total 14 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   created_date            datetime64[ns]
 1   agency                  string        
 2   agency_name             string        
 3   complaint_type          string        
 4   incident_zip            int64         
 5   borough                 string        
 6   community_board         string        
 7   resolution_description  string        
 8   closed_date             datetime64[ns]
 9   due_date                datetime64[ns]
 10  descriptor              string        
 11  median_income_2022      float64       
 12  poverty_rate_2021       float64       
 13  diversity_index_2021    float64       
dtypes: datetime64[ns](3), float64(3), int64(1), string(7)
memory usage: 313.6 MB


In [39]:
req_df.shape

(2936143, 14)

Our processed dataset contains 14 columns, including request information and demographic data.

In [ ]:
#exclude to parquet for easier retrieval in separate notebooks
#req_df.to_parquet('req_df_clean.parquet.gzip', compression='gzip')